<h1 style="font-family: overpass;">Step 1: Paste the key and token info in the following cell and run</h1>

In [ ]:
CONSUMER_KEY = 'insert here'
CONSUMER_SECRET = 'insert here'
ACCESS_TOKEN = 'insert here'
ACCESS_TOKEN_SECRET = 'insert here' 

<h1 style="font-family: overpass;">Step 2: Run the following cell and enter in the search words/phrases as prompted</h1>
<h3 style="font-family: overpass;">Note: See Twitter Advanced Search documentation for more info</h3>

In [ ]:
print("Search String Entry\n")
search_string_all = raw_input("All of these words: ")
search_string_exact = raw_input("This exact phrase: ")
search_string_any = raw_input("Any of these words: ")
search_string_none = raw_input("None of these words: ")

# This is used in case the following cell is accidentally run more than once!
search_is_new = True

<h1 style="font-family: overpass;">Step 3:  Run each of the following cells in sequence to get the results!</h1>

In [ ]:
if search_is_new: 
    if search_string_exact: 
        search_string_exact = '\"' + search_string_exact + '\"'
    if search_string_any: 
        search_string_any = search_string_any.replace(" ", " OR ")
    if search_string_none: 
        search_string_none = "-" + search_string_none.replace(" ", " -")
    
    search_string_list = [search_string_all, search_string_exact, search_string_any, search_string_none]
    search_string_list = filter(None, search_string_list)
    search_string = " ".join(search_string_list)
    
    # Now that we've done this bit, we don't want to accidentally overdo it if we run this cell twice or more!
    search_is_new = False

print("Your search string is: \'" + search_string + "\'")

In [ ]:
import twitter
import tweepy
from datetime import datetime
import time
import re
import sys
import pandas as pd
import pyprind as pp

"""ts = twitter.Twitter(auth=twitter.OAuth(ACCESS_TOKEN, 
                ACCESS_TOKEN_SECRET, CONSUMER_KEY, 
                CONSUMER_SECRET))

def isAuth():
    return bool(isinstance(ts, twitter.api.Twitter))

print('Authentification successful: %s' %isAuth())"""

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)


api = tweepy.API(auth)

tweets_txt = []
for tweet in tweepy.Cursor(api.search,
                           q=search_string,
                           count=100,
                           result_type="recent",
                           include_entities=True,
                           lang="en").items():
    tweets_txt.append(tweet.text)

In [ ]:
print("The search resulted in %s tweets"  %len(tweets_txt))

In [ ]:
tweets_txt

In [ ]:
# Used for pulling overpass font if necessary
import urllib
urllib.urlretrieve("https://github.com/RedHatBrand/Overpass/releases/download/3.0.2/overpass-desktop-fonts.zip", "overpass.zip")

import zipfile
zip_ref = zipfile.ZipFile('./overpass.zip', 'r')
zip_ref.extractall('./overpass_font/')
zip_ref.close()

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

# Gather all the tweets together
words = ' '.join(tweets_txt)

# Remove URLs, RTs, and Twitter handles
no_urls_no_tags = " ".join([word for word in words.split()
                            if 'http' not in word
                                and not word.startswith('@')
                                and word != 'RT'
                            ])

wordcloud = WordCloud(
                      font_path='./overpass_font/overpass/overpass-regular.otf',
                      stopwords=STOPWORDS,
                      background_color='white',
                      width=1800,
                      height=1400
                     ).generate(no_urls_no_tags)

import random

def rh_color_func(word, font_size, position, orientation, random_state=None, **kwargs):
    cols = ["hsl(0, 100%, 40%)", "hsl(0, 100%, 32%)", "hsl(0, 100%, 25%)", "hsl(0, 0%, 0%)"]
    return random.choice(cols)

wc = wordcloud.recolor(color_func=rh_color_func, random_state=3)

fig = plt.figure(figsize = (10,10))
ax = fig.add_subplot(111)
ax.imshow(wc)
plt.axis('off')
'''plt.savefig('my_twitter_wordcloud_1.png', dpi=300)'''
plt.show()

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
for tweet in tweets_txt:
    vs = analyzer.polarity_scores(tweet)
    print("{:-<65} {}".format(tweet.encode("utf-8"), str(vs)))

In [ ]:
sentscores = []
for tweet in tweets_txt:
    vs = analyzer.polarity_scores(tweet)
    sentscores.append(vs['compound'])

import numpy as np

print('Mean score: %5.3f' %np.mean(sentscores))